In [105]:
import sys
sys.path.append("/Users/peterjmyers/Documents/Python_Modules")
import re
import pandas as pd
import pandasql as ps
import numpy as np
from file_util.file_util import *
from sql_util.sql_util import *

df = pd.read_csv('./train.csv')

def add_has_letter_in_ticket_to_df():
    has_letter_in_ticket = []
    for index, row in df.iterrows():
        variable = re.match('.*([A-Za-z]).*', str(row['Ticket']))
        if variable == None:
            has_letter_in_ticket.append(0)
        else:
            has_letter_in_ticket.append(1)
    df['has_letter_in_ticket'] = has_letter_in_ticket
    
add_has_letter_in_ticket_to_df()    
clean_df = get_df_from_local_query('./sql/clean1.sql', df)
# save_df_to_excel('debugging', clean_df)
print("Done")

Done


In [106]:
def add_three_features_to_df_based_on_scrabble_word_score():
    SCORES = {
      'a' : 1, 'b' : 3, 'c' : 3 , 'd' : 2, 'e' : 1, 'f' : 4, 'g' : 2,
      'h' : 4, 'i' : 1, 'j' : 8 , 'k' : 5, 'l' : 1, 'm' : 3, 'n' : 1,
      'o' : 1, 'p' : 3, 'q' : 10, 'r' : 1, 's' : 1, 't' : 1, 'u' : 1,
      'v' : 4, 'w' : 4, 'x' : 8 , 'y' : 4, 'z' : 10,
    }
    first_letter_of_last_name_scrabble_score = []
    last_name_scrabble_score = []
    avg_scrabble_letter_score_in_last_name = []
    for index, row in clean_df.iterrows():
        total_score = 0
        word_length = 0
        for letter in row['last_name']:
            letter_score = SCORES.get(letter.lower(),0)
            if total_score == 0:
                first_letter_of_last_name_scrabble_score.append(letter_score)
            if letter_score != 0:
                word_length = word_length + 1
            total_score = total_score + letter_score
        
        if word_length == 0:
            avg_scrabble_letter_score_in_last_name.append(0)
        else:
            avg_scrabble_letter_score_in_last_name.append(total_score / word_length)
        
        last_name_scrabble_score.append(total_score)
    
    clean_df['first_letter_of_last_name_scrabble_score'] = first_letter_of_last_name_scrabble_score
    clean_df['last_name_scrabble_score'] = last_name_scrabble_score
    clean_df['avg_scrabble_letter_score_in_last_name'] = avg_scrabble_letter_score_in_last_name
        
        
#         variable = re.match('.*([A-Za-z]).*', str(row['Ticket']))
#         if variable == None:
#             has_letter_in_ticket.append(0)
#         else:
#             has_letter_in_ticket.append(1)
#     df['has_letter_in_ticket'] = has_letter_in_ticket
    
        
add_three_features_to_df_based_on_scrabble_word_score()
clean_df = clean_df.drop(['last_name'], axis=1)
save_df_to_excel('debugging', clean_df)
print("Done")


Done


## TPOT Training

In [81]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import numpy as np


X_train, X_test, y_train, y_test = train_test_split(clean_df.drop(['Survived'], axis=1),
                                                    clean_df['Survived'], train_size=0.75, test_size=0.25)
# print(X_train, X_test, y_train, y_test)

tpot = TPOTClassifier(max_time_mins=40, population_size=100, max_eval_time_mins=5, verbosity=2)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))
tpot.export('tpot_pipeline.py')

/Users/peterjmyers/anaconda/envs/root3/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMulti' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/Users/peterjmyers/anaconda/envs/root3/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
Optimization Progress: 100%|██████████| 200/200 [01:24<00:00,  1.70pipeline/s]

Generation 1 - Current best internal CV score: 0.8233156689401779


Optimization Progress: 100%|██████████| 300/300 [02:26<00:00,  1.48pipeline/s]

Generation 2 - Current best internal CV score: 0.8233156689401779


Optimization Progress: 100%|██████████| 400/400 [03:59<00:00,  1.27pipeline/s]

...

Generation 16 - Current best internal CV score: 0.8338640963935544


Optimization Progress: 100%|██████████| 1800/1800 [38:03<00:00,  1.07s/pipeline]

Generation 17 - Current best internal CV score: 0.8338640963935544



40.030132683333335 minutes have elapsed. TPOT will close down.
TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: ExtraTreesClassifier(DecisionTreeClassifier(input_matrix, criterion=entropy, max_depth=1, min_samples_leaf=19, min_samples_split=18), bootstrap=False, criterion=gini, max_features=1.0, min_samples_leaf=1, min_samples_split=8, n_estimators=100)
0.807174887892


True

In [107]:
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.tree import DecisionTreeClassifier
from tpot.builtins import StackingEstimator

# NOTE: Make sure that the class is labeled 'target' in the data file
clean_df = clean_df.drop(['PassengerId'], axis=1)
features = clean_df.drop('Survived', axis=1).values
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, clean_df['Survived'].values, random_state=42)

# Score on the training set was:0.8338640963935544
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=DecisionTreeClassifier(criterion="entropy", max_depth=1, min_samples_leaf=19, min_samples_split=18)),
    ExtraTreesClassifier(bootstrap=False, criterion="gini", max_features=1.0, min_samples_leaf=1, min_samples_split=8, n_estimators=100)
)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

print("Done")

# test_df = pd.read_csv('./test.csv')

Done


In [108]:
from sklearn.metrics import confusion_matrix
print(sum(results))
confusion_matrix(testing_target, results)

# training_features, , training_target, 


88


array([[112,  22],
       [ 23,  66]])

## Create Data for Submission

In [143]:
import sys
sys.path.append("/Users/peterjmyers/Documents/Python_Modules")
import re
import pandas as pd
import pandasql as ps
import numpy as np
from file_util.file_util import *
from sql_util.sql_util import *
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.tree import DecisionTreeClassifier
from tpot.builtins import StackingEstimator

df = pd.read_csv('./test.csv')


Survived = [0] * len(df)
df['Survived'] = Survived
add_has_letter_in_ticket_to_df()    

clean_df = get_df_from_local_query('./sql/clean1.sql', df)
add_three_features_to_df_based_on_scrabble_word_score()
clean_df = clean_df.drop(['last_name'], axis=1)
clean_df = clean_df.drop(['Survived'], axis=1)

predict_df = clean_df.drop(['PassengerId'], axis=1)

save_df_to_excel('debugging2', clean_df)

results = exported_pipeline.predict(predict_df.values)

#clean_df.describe()
#print("Done")
#clean_df.head()
# test_df = pd.read_csv('./test.csv')

In [152]:

submit_df = clean_df['PassengerId']

print(type(submit_df))
print(type(results))

results_df = pd.DataFrame(results)

save_df_to_excel('submit', clean_df)

df.to_csv('pathToCSV')
    
pd.concat([submit_df, results_df], axis=1)

<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>


,PassengerId,0
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
5,897,0
6,898,1
7,899,0
8,900,1
9,901,0


In [117]:
save_df_to_excel('debugging', testing_features)

AttributeError: 'numpy.ndarray' object has no attribute 'to_excel'

In [139]:
#numpy.ndarray
print(testing_features.dtype)
print(testing_features.shape)
np.isnan(testing_features)
np.where(np.isnan(testing_features))

float64
(223, 20)


(array([], dtype=int64), array([], dtype=int64))

In [138]:

print(clean_df.values.dtype)
print(clean_df.values.shape)
np.isnan(clean_df.values)
np.where(np.isnan(clean_df.values))

float64
(418, 20)


(array([], dtype=int64), array([], dtype=int64))

In [127]:
file = open("./debug3","w")
line = file.write(testing_features)
file.close()

TypeError: write() argument must be str, not numpy.ndarray